In [1]:
import sys
import os
current_dir = os.getcwd()  #use to import the defined functions
parent_dir = os.path.dirname(current_dir) 
sys.path.append(parent_dir)  

"""
incase the above code does not work, you can use the absolute path instead
sys.path.append(r".\")
"""


'\nincase the above code does not work, you can use the absolute path instead\nsys.path.append(r".")\n'

In [2]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau 

In [3]:
from utils.basic_utils import setup_seed
from data.SimulationData import DataGenerator 
from utils.training_utils import train_WGR_fnn
from utils.evaluation_utils import L1L2_MSE_mean_sd_G, MSE_quantile_G_uniY
from models.generator import generator_fnn
from models.discriminator import discriminator_fnn

In [4]:
import argparse

if 'ipykernel_launcher.py' in sys.argv[0]:  #if not work in jupyter, you can delete this part
    import sys
    sys.argv = [sys.argv[0]] 


parser = argparse.ArgumentParser(description='Implementation of WGR for M1')

parser.add_argument('--Xdim', default=100, type=int, help='dimensionality of X')
parser.add_argument('--Ydim', default=1, type=int, help='dimensionality of Y')
parser.add_argument('--model', default='M1', type=str, help='model')

parser.add_argument('--noise_dim', default=5, type=int, help='dimensionality of noise vector')
parser.add_argument('--noise_dist', default='gaussian', type=str, help='distribution of noise vector')

parser.add_argument('--train', default=5000, type=int, help='size of train dataset')
parser.add_argument('--val', default=1000, type=int, help='size of validation dataset')
parser.add_argument('--test', default=1000, type=int, help='size of test dataset')

parser.add_argument('--train_batch', default=128, type=int, metavar='BS', help='batch size while training')
parser.add_argument('--val_batch', default=100, type=int, metavar='BS', help='batch size while validation')
parser.add_argument('--test_batch', default=100, type=int, metavar='BS', help='batch size while testing')
parser.add_argument('--epochs', default=100, type=int, help='number of epochs to train')
parser.add_argument('--reps', default=100, type=int, help='number of replications')

args = parser.parse_args()

print(args)

Namespace(Xdim=100, Ydim=1, model='M1', noise_dim=5, noise_dist='gaussian', train=5000, val=1000, test=1000, train_batch=128, val_batch=100, test_batch=100, epochs=100, reps=100)


In [5]:
# Set seed 
setup_seed(1235)

In [6]:
# Generate data from M1
data_gen = DataGenerator(args)
DATA = data_gen.generate_data(args.model)
train_X, train_Y = DATA['train_X'], DATA['train_Y']
val_X, val_Y = DATA['val_X'], DATA['val_Y']
test_X, test_Y = DATA['test_X'], DATA['test_Y']

In [7]:
# Create TensorDatasets and initialize a DataLoaders
train_dataset = TensorDataset( train_X.float(), train_Y.float() )
loader_train = DataLoader(train_dataset , batch_size=args.train_batch, shuffle=True)

val_dataset = TensorDataset( val_X.float(), val_Y.float() )
loader_val = DataLoader(val_dataset , batch_size=args.val_batch, shuffle=True)

test_dataset = TensorDataset( test_X.float(), test_Y.float() )
loader_test  = DataLoader(test_dataset , batch_size=args.test_batch, shuffle=True)

In [8]:
# Define generator network and discriminator network
G_net = generator_fnn(Xdim=args.Xdim, Ydim=args.Ydim, noise_dim=args.noise_dim, hidden_dims = [64, 32])
D_net = discriminator_fnn(input_dim=args.Xdim+args.Ydim, hidden_dims = [64, 32])

In [9]:
# Initialize RMSprop optimizers
D_solver = optim.RMSprop(D_net.parameters(),lr = 0.001)
G_solver = optim.RMSprop(G_net.parameters(),lr = 0.001)


In [10]:
# Training
trained_G, trained_D = train_WGR_fnn(D=D_net, G=G_net, D_solver=D_solver, G_solver=G_solver, 
                                     loader_train = loader_train, loader_val=loader_val, noise_dim=args.noise_dim, 
                                     Xdim=args.Xdim, Ydim=args.Ydim, batch_size=args.train_batch, 
                                     save_path='./', device='cpu', num_epochs=100)

Mean L1 Loss: 3.254895, Mean L2 Loss: 18.813904
Epoch 0 - D Loss: 0.8040, G Loss: 1.6796
Epoch 1 - D Loss: 0.1826, G Loss: 1.9566
Epoch 2 - D Loss: 0.0151, G Loss: 2.1025
Epoch 3 - D Loss: 0.0942, G Loss: 2.1219
Epoch 4 - D Loss: 0.2083, G Loss: 2.3719
Epoch 5 - D Loss: 0.1377, G Loss: 3.0096
Epoch 6 - D Loss: 0.0252, G Loss: 3.5932
Epoch 7 - D Loss: 0.0213, G Loss: 3.7865
Epoch 8 - D Loss: 0.0141, G Loss: 3.7014
Epoch 9 - D Loss: 0.0015, G Loss: 3.5755
Epoch 10 - D Loss: -0.0007, G Loss: 3.4309
Epoch 11 - D Loss: -0.0072, G Loss: 3.4327
Epoch 12 - D Loss: -0.0107, G Loss: 3.4724
Epoch 13 - D Loss: -0.0168, G Loss: 3.4260
Epoch 14 - D Loss: -0.0194, G Loss: 3.3356
Epoch 15 - D Loss: -0.0170, G Loss: 3.2924
Epoch 16 - D Loss: -0.0239, G Loss: 3.2110
Epoch 17 - D Loss: -0.0221, G Loss: 3.1438
Epoch 18 - D Loss: -0.0224, G Loss: 3.0853
Epoch 19 - D Loss: -0.0259, G Loss: 3.0653
Epoch 20 - D Loss: -0.0229, G Loss: 2.9922
Epoch 21 - D Loss: -0.0192, G Loss: 2.9663
Epoch 22 - D Loss: -0.0216

In [11]:
# Calculate the L1 and L2 error, MSE of conditional mean and conditional standard deviation on the test data  
test_G_mean_sd = L1L2_MSE_mean_sd_G(G = trained_G,  test_size = args.test, noise_dim=args.noise_dim, Xdim=args.Xdim,
                                    batch_size=args.test_batch,  model_type=args.model, loader_dataset = loader_test )

Model: M1, Univariate, Ydim: 1, J_t_size: 50
L1 Loss: tensor([1.6839])
L2 Loss: tensor([5.3885])
MSE Mean: tensor([2.5918])
MSE SD: tensor([0.2850])


In [12]:
# Calculate the MSE of conditional quantiles at different levels.
test_G_quantile = MSE_quantile_G_uniY(G = trained_G, loader_dataset = loader_test , noise_dim=args.noise_dim, Xdim=args.Xdim,
                                      test_size = args.test,  batch_size=args.test_batch, model_type=args.model)

Q_5: 2.9589, Q_25: 2.6305, Q_50: 2.6051, Q_75: 2.8051, Q_95: 3.6236
